In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [3]:
df = pd.read_parquet("../DATA/dataset/TRAIN_stage1")

---
# 다중공선성 check

In [14]:
X = df.drop(columns=["fraud"])
corr = X.corr().abs()

In [5]:
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

high_corr_pairs = (
    upper.stack()
    .reset_index()
    .rename(columns={0: "corr"})
    .query("corr >= 0.8")
    .sort_values("corr", ascending=False)
)

high_corr_pairs


,level_0,level_1,corr
47,amount_vs_client_avg_diff,amount_deviation,0.971414
1,log_abs_amount,amount_vs_client_avg_diff,0.919937
2,log_abs_amount,amount_deviation,0.895992
196,card_fraud_last1,card_fraud_last3,0.848580


즉시성 -> last 1 살림\

| 변수                        | 의미                |
| ------------------------- | ----------------- |
| log_abs_amount            | 절대 규모             |
| amount_vs_client_avg_diff | 평균 대비 차이          |
| amount_deviation          | 평균 대비 표준화 z-score |

-> amount_deviation이 제일 정보 압축 + 스케일 안정 

[유지]
- log_abs_amount
- amount_deviation
- card_fraud_last1

[제거]
- amount_vs_client_avg_diff
- card_fraud_last3

In [6]:
df.drop(columns=["amount_vs_client_avg_diff", "card_fraud_last3"], inplace=True)

---
# SHAP

In [7]:
LABEL_COL = "fraud"  

y = df[LABEL_COL].astype(int)
X = df.drop(columns=[LABEL_COL])

# 1) 컬럼 타입 자동 추정

cat_cols = [c for c in X.columns if str(X[c].dtype) in ("object", "category")]
num_cols = [c for c in X.columns if c not in cat_cols]

print("num_cols:", len(num_cols), "cat_cols:", len(cat_cols))


# 2) Train/Valid split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3) 간단 전처리 파이프

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)), 
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
    ],
    remainder="drop",
)


num_cols: 23 cat_cols: 0


In [8]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import shap
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm.auto import tqdm

X_train_lgb = X_train.copy()
X_valid_lgb = X_valid.copy()

dtrain = lgb.Dataset(X_train_lgb, label=y_train, free_raw_data=False)
dvalid = lgb.Dataset(X_valid_lgb, label=y_valid, free_raw_data=False)

params = dict(
    objective="binary",
    metric=["auc", "average_precision"],
    learning_rate=0.05,
    num_leaves=64,
    min_data_in_leaf=200,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    verbosity=-1,
)

bst = lgb.train(
    params,
    dtrain,
    num_boost_round=5000,
    valid_sets=[dvalid],
    valid_names=["valid"],
    callbacks=[
        lgb.early_stopping(200, verbose=True),
        lgb.log_evaluation(0),
    ],
)

print("best_iter:", bst.best_iteration)
print("best_score:", bst.best_score)

pred_valid = bst.predict(X_valid_lgb, num_iteration=bst.best_iteration)
print("LGB AUC:", roc_auc_score(y_valid, pred_valid))
print("LGB PR-AUC:", average_precision_score(y_valid, pred_valid))


# SHAP with tqdm (batch version)

from tqdm.auto import tqdm

sv = X_valid_lgb

batch_size = 2000
all_contrib = []

print("\nComputing SHAP (LightGBM native)...")

for i in tqdm(range(0, len(sv), batch_size)):
    batch = sv.iloc[i:i+batch_size]
    contrib = bst.predict(batch, pred_contrib=True)
    all_contrib.append(contrib[:, :-1])  # 마지막 열은 bias

shap_values = np.vstack(all_contrib)

imp = pd.Series(
    np.abs(shap_values).mean(axis=0),
    index=sv.columns
).sort_values(ascending=False)

print("\nTop SHAP features:\n", imp.head(30))


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[41]	valid's auc: 0.930985	valid's average_precision: 0.573383
best_iter: 41
best_score: defaultdict(<class 'collections.OrderedDict'>, {'valid': OrderedDict([('auc', np.float64(0.9309853858465273)), ('average_precision', np.float64(0.5733826590611859))])})
LGB AUC: 0.9309862609584562
LGB PR-AUC: 0.575246858541035

Computing SHAP (LightGBM native)...


  0%|          | 0/534 [00:00<?, ?it/s]


Top SHAP features:
 client_fraud_last1             2.213367
seconds_since_prev_tx          1.357237
tx_hour                        1.076400
card_velocity_spike_ratio      0.877179
amount_deviation               0.721085
mcc_highrisk_90                0.285328
client_merchant_is_new         0.189004
log_abs_amount                 0.086074
card_fraud_last1               0.076221
card_merchant_is_new           0.044432
hour_cos                       0.023145
is_highrisk_weekday            0.018885
tx_month                       0.013154
card_mcc_is_new                0.008688
client_mcc_is_new              0.003291
high_amount                    0.001729
err_bad_cvv                    0.001469
has_error                      0.001329
client_error_last1             0.000745
err_bad_card_number            0.000536
err_bad_expiration             0.000421
card_error_last1               0.000180
merchant_is_new_x_has_error    0.000036
dtype: float64


---
# Attention 

In [9]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score

feature_names = list(X_train.columns)
n_features = len(feature_names)

scaler = StandardScaler()
Xtr = scaler.fit_transform(X_train.values.astype(np.float32))
Xva = scaler.transform(X_valid.values.astype(np.float32))

ytr = y_train.values.astype(np.float32)
yva = y_valid.values.astype(np.float32)

class TabDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_loader = DataLoader(TabDataset(Xtr, ytr), batch_size=4096, shuffle=True, num_workers=0)
valid_loader = DataLoader(TabDataset(Xva, yva), batch_size=8192, shuffle=False, num_workers=0)

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Attention layer (weights 반환)

class AttnEncoderLayer(nn.Module):
    def __init__(self, d_model=64, n_heads=4, dropout=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.ln1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 4, d_model),
            nn.Dropout(dropout),
        )
        self.ln2 = nn.LayerNorm(d_model)

    def forward(self, x, return_attn=False):
        # x: [B, T, D]
        attn_out, attn_w = self.mha(x, x, x, need_weights=True, average_attn_weights=False)
        x = self.ln1(x + attn_out)
        x = self.ln2(x + self.ff(x))
        if return_attn:
            # attn_w: [B, heads, T, T]
            return x, attn_w
        return x


# 2) Tabular Transformer (CLS 토큰 사용)

class TabularAttentionModel(nn.Module):
    def __init__(self, n_features, d_model=64, n_heads=4, n_layers=2, dropout=0.1):
        super().__init__()
        self.n_features = n_features
        self.d_model = d_model

        # feature별 1->d 투영 (각 feature마다 별도의 linear)
        self.feat_proj = nn.ModuleList([nn.Linear(1, d_model) for _ in range(n_features)])

        # CLS token
        self.cls = nn.Parameter(torch.zeros(1, 1, d_model))
        nn.init.normal_(self.cls, std=0.02)

        self.layers = nn.ModuleList([AttnEncoderLayer(d_model, n_heads, dropout) for _ in range(n_layers)])

        self.head = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, 1),
        )

    def forward(self, X, return_attn=False):
        # X: [B, F]
        B, F = X.shape

        # feature tokens 만들기: [B, F, D]
        toks = []
        for j in range(F):
            xj = X[:, j:j+1]                 # [B, 1]
            toks.append(self.feat_proj[j](xj))  # [B, D]
        tok = torch.stack(toks, dim=1)       # [B, F, D]

        # CLS 붙이기: [B, 1+F, D]
        cls = self.cls.expand(B, -1, -1)
        x = torch.cat([cls, tok], dim=1)

        attn_all = []
        for layer in self.layers:
            if return_attn:
                x, attn = layer(x, return_attn=True)
                attn_all.append(attn)
            else:
                x = layer(x, return_attn=False)

        # CLS representation으로 예측
        cls_repr = x[:, 0, :]               # [B, D]
        logit = self.head(cls_repr).squeeze(1)

        if return_attn:
            return logit, attn_all  # list of [B, heads, T, T]
        return logit


# 3) 학습 루프

model = TabularAttentionModel(n_features=n_features, d_model=64, n_heads=4, n_layers=2, dropout=0.1).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
loss_fn = nn.BCEWithLogitsLoss()

def eval_model():
    model.eval()
    ps, ys = [], []
    with torch.no_grad():
        for xb, yb in valid_loader:
            xb = xb.to(device)
            logit = model(xb)
            prob = torch.sigmoid(logit).cpu().numpy()
            ps.append(prob)
            ys.append(yb.numpy())
    p = np.concatenate(ps)
    t = np.concatenate(ys)
    return roc_auc_score(t, p), average_precision_score(t, p)

EPOCHS = 5
for epoch in range(1, EPOCHS + 1):
    model.train()
    pbar = tqdm(train_loader, desc=f"train epoch {epoch}", leave=False)
    for xb, yb in pbar:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad(set_to_none=True)
        logit = model(xb)
        loss = loss_fn(logit, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        pbar.set_postfix(loss=float(loss.detach().cpu()))

    auc, pr = eval_model()
    print(f"[epoch {epoch}] valid AUC={auc:.5f}  PR-AUC={pr:.5f}")


# 4) Attention 추출 → 컬럼 중요도
# - CLS(0번 토큰)에서 각 feature 토큰으로 가는 attention을 사용

def extract_feature_attention_importance(model, loader, n_batches=50):
    model.eval()
    # 누적: feature별 attention 합
    att_sum = np.zeros((n_features,), dtype=np.float64)
    cnt = 0

    with torch.no_grad():
        for b, (xb, yb) in enumerate(tqdm(loader, desc="extract attention", total=min(n_batches, len(loader)))):
            if b >= n_batches:
                break
            xb = xb.to(device)

            logit, attn_all = model(xb, return_attn=True)
            attn = attn_all[-1]  # [B, heads, T, T]

            # CLS -> feature 토큰 attention: query=0, key=1..F
            # shape: [B, heads, F]
            cls_to_feat = attn[:, :, 0, 1:]  

            # heads 평균, batch 평균 → [F]
            score = cls_to_feat.mean(dim=1).mean(dim=0).cpu().numpy()
            att_sum += score
            cnt += 1

    att_mean = att_sum / max(cnt, 1)
    imp = pd.Series(att_mean, index=feature_names).sort_values(ascending=False)
    return imp

att_imp = extract_feature_attention_importance(model, valid_loader, n_batches=50)
print("\nTop Attention features:\n", att_imp.head(30))


train epoch 1:   0%|          | 0/1042 [00:00<?, ?it/s]

[epoch 1] valid AUC=0.96969  PR-AUC=0.66117


train epoch 2:   0%|          | 0/1042 [00:00<?, ?it/s]

[epoch 2] valid AUC=0.97064  PR-AUC=0.68437


train epoch 3:   0%|          | 0/1042 [00:00<?, ?it/s]

...

[epoch 4] valid AUC=0.97282  PR-AUC=0.68471


train epoch 5:   0%|          | 0/1042 [00:00<?, ?it/s]

[epoch 5] valid AUC=0.97274  PR-AUC=0.68993


extract attention:   0%|          | 0/50 [00:00<?, ?it/s]


Top Attention features:
 mcc_highrisk_90                0.153214
amount_deviation               0.122909
tx_hour                        0.099942
client_merchant_is_new         0.068642
hour_cos                       0.045899
card_merchant_is_new           0.045857
is_highrisk_weekday            0.042593
log_abs_amount                 0.034374
client_error_last1             0.033932
merchant_is_new_x_has_error    0.032522
card_error_last1               0.030472
high_amount                    0.030145
card_mcc_is_new                0.029999
err_bad_expiration             0.026969
client_fraud_last1             0.026301
card_fraud_last1               0.025920
err_bad_cvv                    0.022774
card_velocity_spike_ratio      0.022345
tx_month                       0.021643
seconds_since_prev_tx          0.021176
client_mcc_is_new              0.017835
has_error                      0.015020
err_bad_card_number            0.014238
dtype: float64


In [10]:
compare = pd.DataFrame({
    "shap_mean_abs": imp,        
    "attn_cls2feat": att_imp
}).fillna(0.0)

compare["shap_rank"] = compare["shap_mean_abs"].rank(ascending=False, method="min")
compare["attn_rank"] = compare["attn_cls2feat"].rank(ascending=False, method="min")
compare["rank_gap"] = compare["attn_rank"] - compare["shap_rank"]

print(compare.sort_values("shap_mean_abs", ascending=False).head(40))


                             shap_mean_abs  attn_cls2feat  shap_rank  \
client_fraud_last1                2.213367       0.026301        1.0   
seconds_since_prev_tx             1.357237       0.021176        2.0   
tx_hour                           1.076400       0.099942        3.0   
card_velocity_spike_ratio         0.877179       0.022345        4.0   
amount_deviation                  0.721085       0.122909        5.0   
mcc_highrisk_90                   0.285328       0.153214        6.0   
client_merchant_is_new            0.189004       0.068642        7.0   
log_abs_amount                    0.086074       0.034374        8.0   
card_fraud_last1                  0.076221       0.025920        9.0   
card_merchant_is_new              0.044432       0.045857       10.0   
hour_cos                          0.023145       0.045899       11.0   
is_highrisk_weekday               0.018885       0.042593       12.0   
tx_month                          0.013154       0.021643       

---
# Ablation

In [11]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm.auto import tqdm

params = dict(
    objective="binary",
    metric="auc",
    learning_rate=0.05,
    num_leaves=64,
    min_data_in_leaf=200,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    verbosity=-1,
)

def train_eval(X_tr, y_tr, X_va, y_va):
    dtrain = lgb.Dataset(X_tr, label=y_tr, free_raw_data=False)
    dvalid = lgb.Dataset(X_va, label=y_va, free_raw_data=False)

    bst = lgb.train(
        params,
        dtrain,
        num_boost_round=3000,
        valid_sets=[dvalid],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)],
    )

    pred = bst.predict(X_va, num_iteration=bst.best_iteration)

    return {
        "auc": roc_auc_score(y_va, pred),
        "prauc": average_precision_score(y_va, pred),
        "best_iter": bst.best_iteration,
    }


In [12]:
base_result = train_eval(X_train, y_train, X_valid, y_valid)
print("BASE:", base_result)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[41]	valid_0's auc: 0.930985
BASE: {'auc': 0.9309862609584562, 'prauc': 0.575246858541035, 'best_iter': 41}


In [13]:
drop_one_results = []

for col in tqdm(X_train.columns):
    cols = [c for c in X_train.columns if c != col]

    res = train_eval(
        X_train[cols],
        y_train,
        X_valid[cols],
        y_valid,
    )

    drop_one_results.append({
        "dropped_feature": col,
        "auc_drop": base_result["auc"] - res["auc"],
        "prauc_drop": base_result["prauc"] - res["prauc"],
        "auc": res["auc"],
        "prauc": res["prauc"],
    })

drop_one_df = pd.DataFrame(drop_one_results)\
    .sort_values("auc_drop", ascending=False)

drop_one_df

  0%|          | 0/23 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.95168
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.951737
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.9519
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.951737
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.951737
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.951737
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.951737
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.951737
Tra

,dropped_feature,auc_drop,prauc_drop,auc,prauc
17,mcc_highrisk_90,-0.013567,-0.080113,0.944553,0.655360
9,card_fraud_last1,-0.019567,0.046422,0.950553,0.528825
0,log_abs_amount,-0.020694,-0.079517,0.951680,0.654764
10,client_fraud_last1,-0.020750,-0.080199,0.951736,0.655445
11,tx_hour,-0.020750,-0.080199,0.951736,0.655445
1,high_amount,-0.020751,-0.080193,0.951737,0.655440
3,has_error,-0.020751,-0.080193,0.951737,0.655440
7,card_error_last1,-0.020751,-0.080193,0.951737,0.655440
6,err_bad_expiration,-0.020751,-0.080193,0.951737,0.655440
4,err_bad_cvv,-0.020751,-0.080193,0.951737,0.655440


---

# Stage1 Feature Selection Summary

(SHAP + Attention + Ablation 종합 정리)

---

## 1. Strong Keep (핵심 유지)

| Feature                     | 근거                            |
| --------------------------- | ----------------------------- |
| `client_fraud_last1`        | SHAP 1위, 모델 의존도 매우 높음         |
| `seconds_since_prev_tx`     | SHAP 2위, 실시간 이상 패턴 포착         |
| `tx_hour`                   | SHAP/Attention 상위             |
| `card_velocity_spike_ratio` | Velocity 신호, ablation 영향 큼    |
| `amount_deviation`          | Amount anomaly 핵심             |
| `mcc_highrisk_90`           | Attention 1위, 제거 시 AUC 감소     |
| `client_merchant_is_new`    | SHAP/Attention/ablation 모두 중요 |

이들은 Stage1 리스크 구조의 중심 축.

---

## 2. Keep (유지 권장)

| Feature                | 근거                 |
| ---------------------- | ------------------ |
| `log_abs_amount`       | 기본 금액 강도           |
| `card_fraud_last1`     | 카드 단위 과거 리스크       |
| `card_merchant_is_new` | 신규 상점 신호           |
| `hour_cos`             | 시간 패턴 보조           |
| `is_highrisk_weekday`  | 요일 리스크 보조          |
| `tx_month`             | 계절성 보조             |
| `card_mcc_is_new`      | 신규 MCC             |
| `client_mcc_is_new`    | 신규 MCC (client 기준) |

보조적이지만 구조적 의미가 있음.

---

## 3. Drop 후보 (기여도 낮음)

| Feature                       | 근거         |
| ----------------------------- | ---------- |
| `high_amount`                 | SHAP 매우 낮음 |
| `err_bad_cvv`                 | 영향 미미      |
| `has_error`                   | 단독 기여 낮음   |
| `client_error_last1`          | 기여도 매우 낮음  |
| `err_bad_card_number`         | 낮음         |
| `err_bad_expiration`          | 낮음         |
| `card_error_last1`            | 거의 영향 없음   |
| `merchant_is_new_x_has_error` | 영향 거의 없음   |

제거해도 성능 변화가 거의 없을 가능성 높음.

---

## 4. Stage1 구조 해석

현재 Stage1 모델은 다음 6개 축으로 구성됨:

1. Fraud history
2. Velocity
3. Time
4. MCC high-risk
5. Merchant novelty
6. Amount anomaly

Error 세부 변수들은 Stage2로 넘기는 것이 구조적으로 적절.

---

## 5. Slim Stage1 제안 (가볍게)

```python
CORE_STAGE1 = [
    "log_abs_amount",
    "amount_deviation",
    "client_fraud_last1",
    "card_fraud_last1",
    "seconds_since_prev_tx",
    "card_velocity_spike_ratio",
    "tx_hour",
    "hour_cos",
    "mcc_highrisk_90",
    "client_merchant_is_new",
    "card_merchant_is_new",
]
```

이 구성은:

* 실시간성 유지
* 계산 부담 최소화
* 구조적 리스크 축 유지
* 중복 신호 제거

Stage1을 “빠르고 가볍게” 유지하면서도 성능을 크게 훼손하지 않을 가능성이 높음.

---